In [1]:
# Those steps are forked from https://github.com/hechmik/voxceleb_enrichment_age_gender/tree/main/notebooks

In [1]:
# audio data downloaded from https://www.robots.ox.ac.uk/~vgg/data/voxceleb/

In [2]:
! export KALDI_DIR=/home/dorota/research/kaldi


In [3]:
import kaldi
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import sys
sys.path.append("asvtorch")
sys.path.append("asvtorch/asvtorch")

from src.utterances.utterance_list import UtteranceList
import numpy as np
import itertools
import pandas as pd

In [4]:
train_metadata = pd.read_csv('age-train.txt')
test_metadata = pd.read_csv('age-test.txt')

train_metadata['vox_yt_ids'] = train_metadata['VoxCeleb_ID'] + '-' + train_metadata['video_id']
train_vox_yt = list(train_metadata.vox_yt_ids)

test_metadata['vox_yt_ids'] = test_metadata['VoxCeleb_ID'] + '-' + test_metadata['video_id']
test_vox_yt = list(test_metadata.vox_yt_ids)

len(train_vox_yt), len(test_vox_yt)

(1690, 1127)

In [5]:
folder = "/media/dorota/data2024/voxceleb_outputs/system1/utterances"
label = "trial_embeddings_backup"
plda_data = UtteranceList.load(label, folder)

Loading: /media/dorota/data2024/voxceleb_outputs/system1/utterances/trial_embeddings_backup.pickle
Loaded (11.503 s): /media/dorota/data2024/voxceleb_outputs/system1/utterances/trial_embeddings_backup.pickle


In [6]:
train = []
test = []
train_indexes = []
plda_vox_yt_id = []
test_indexes = []
test_plda_vox_yt_id = []
for i, voxID_video_id in enumerate(plda_data.get_utt_labels()):
    # Let's now remove the "recording" info from voxID-YT id
    current_id = "-".join(voxID_video_id.split("-")[0:-1])

    d={}
    d["voxID_video_id"]=voxID_video_id
    
    idx = voxID_video_id.rfind("-")
    id_us_vid = voxID_video_id[:idx]
    
    d["VoxCeleb_ID"]=voxID_video_id.split("-")[0]
    d["video_id"]=voxID_video_id.split("-")[1]
    d["recording"]=voxID_video_id.split("-")[2]
    
    idx1 = voxID_video_id.find("-")
    p1=voxID_video_id[idx1+1:]
    idx2 = p1.rfind("-")
    p1[:idx2]
    
    path = voxID_video_id.split("-")[0] +"/" + p1[:idx2] + "/" +  voxID_video_id[idx+1:] +".m4a"
    
    file_id = "/home/dorota/research/voxceleb2_joined/dev/aac/"+path
    d["path"]=file_id
    sp_vid = "-".join(voxID_video_id.split("-")[:-1])
    if current_id in train_vox_yt:
        d["subset"]="train"
        
        speaker_age = train_metadata[train_metadata['vox_yt_ids'] == id_us_vid].speaker_age.item()
        d["age"]=speaker_age
        train.append(d)
        plda_vox_yt_id.append(current_id)
    elif current_id in test_vox_yt:
        d["subset"]="test"
        speaker_age = test_metadata[test_metadata['vox_yt_ids'] == id_us_vid].speaker_age.item()
        d["age"]=speaker_age
        test.append(d)
        test_plda_vox_yt_id.append(current_id) 

In [7]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [8]:
train.shape, test.shape

((12722, 7), (8602, 7))

In [9]:
len(plda_vox_yt_id), len(test_plda_vox_yt_id)

(12722, 8602)

In [10]:
train.head()

,voxID_video_id,VoxCeleb_ID,video_id,recording,path,subset,age
0,id08110-jpyddhVD_os-00157,id08110,jpyddhVD_os,00157,/home/dorota/research/voxceleb2_joined/dev/aac...,train,40.0
1,id02273-IjorVdJcHfs-00008,id02273,IjorVdJcHfs,00008,/home/dorota/research/voxceleb2_joined/dev/aac...,train,62.0
2,id08699-SuDPOMMK124-00152,id08699,SuDPOMMK124,00152,/home/dorota/research/voxceleb2_joined/dev/aac...,train,45.0
3,id09049-O2DWjBBML0o-00144,id09049,O2DWjBBML0o,00144,/home/dorota/research/voxceleb2_joined/dev/aac...,train,43.0
4,id01617-LjDFACM2G0E-00104,id01617,LjDFACM2G0E,00104,/home/dorota/research/voxceleb2_joined/dev/aac...,train,36.0


In [11]:
test.head()

,voxID_video_id,VoxCeleb_ID,video_id,recording,path,subset,age
0,id06726-s2lysJWkjus-00184,id06726,s2lysJWkjus,00184,/home/dorota/research/voxceleb2_joined/dev/aac...,test,66.0
1,id02004-yaL6gPvSDe8-00341,id02004,yaL6gPvSDe8,00341,/home/dorota/research/voxceleb2_joined/dev/aac...,test,68.0
2,id05660-YCTHKUYmAqU-00049,id05660,YCTHKUYmAqU,00049,/home/dorota/research/voxceleb2_joined/dev/aac...,test,78.0
3,id05660-YCTHKUYmAqU-00047,id05660,YCTHKUYmAqU,00047,/home/dorota/research/voxceleb2_joined/dev/aac...,test,78.0
4,id00032-ExrjgBxENmo-00042,id00032,ExrjgBxENmo,00042,/home/dorota/research/voxceleb2_joined/dev/aac...,test,35.0


In [12]:
train.to_csv("train.csv")

In [13]:
test.to_csv("test.csv")